In [99]:
import pandas as pd
import numpy as np 
import matplotlib as plt

from scipy.sparse import hstack, csr_matrix


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [ ]:
df = pd.read_csv("C:/Users/Mail Moolbaes/Desktop/Scoo/CSC 316/housing.csv")

In [ ]:
df.columns

Index(['brokered_by', 'status', 'price', 'bed', 'bath', 'acre_lot', 'street',
       'city', 'state', 'zip_code', 'house_size', 'prev_sold_date'],
      dtype='object')


Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus'],
      dtype='object')

In [ ]:
# Define custom order of ordianl variables
custom_order = [['unfurnished', 'semi-furnished', 'furnished']]

# Create OrdinalEncoder with custom categories
enc = OrdinalEncoder(categories=custom_order)

# Fit and transform the column
df[['furnishingstatus']] = enc.fit_transform(df[['furnishingstatus']])

# Define mapping
mapping = {'yes': 1, 'no': 0}

# Replace yes/no in ALL columns at once
df = df.replace(mapping)
df

C:\Users\Mail Moolbaes\AppData\Local\Temp\ipykernel_15536\30907660.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(mapping)


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2.0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2.0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1.0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2.0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,1,0,1,0,0,2,0,0.0
541,1767150,2400,3,1,1,0,0,0,0,0,0,0,1.0
542,1750000,3620,2,1,1,1,0,0,0,0,0,0,0.0
543,1750000,2910,3,1,1,0,0,0,0,0,0,0,2.0


In [ ]:
# feature/target split
y = df['price']
x = df.drop(['price'], axis=1)

In [ ]:
# train/test split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.8, random_state = 42)

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

In [ ]:
# predict
y_pred = model.predict(x_test)

# model metrics
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

mse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("Root Mean Squared Error:", mse)
print("R² Score:", r2)

In [116]:
df2 = pd.read_csv("C:/Users/Mail Moolbaes/Desktop/Scoo/CSC 316/realtor-data.csv")
df2 = df2.dropna()

# unique
for col in df2:
    print(f'the number of unique values in {col}: {df2[col].nunique()}')

the number of unique values in brokered_by: 82809
the number of unique values in status: 2
the number of unique values in price: 37948
the number of unique values in bed: 70
the number of unique values in bath: 62
the number of unique values in acre_lot: 6165
the number of unique values in street: 1003765
the number of unique values in city: 15103
the number of unique values in state: 53
the number of unique values in zip_code: 24855
the number of unique values in house_size: 10329
the number of unique values in prev_sold_date: 14081


In [ ]:
# feature/target split
x = df2.drop(['price'], axis=1)
y = df2['price']

# sparse matrix from categorical
encoder = OneHotEncoder(dtype=int)
df_encoded_sparse = encoder.fit_transform(x[['brokered_by', 'status', 'street', 'city', 'state', 'zip_code', 'bed', 'bath']]) # prev_sold_date not helpful  

# sparse matrix from numeric
num_array_sparse = x[['acre_lot', 'house_size']].values # 'bed', 'bath' better as categorical 
num_array_sparse = csr_matrix(num_array_sparse)

final_sparse = hstack([num_array_sparse, df_encoded_sparse])

In [ ]:
print(df2.size, df2.shape)

# size on sparse only includes stored values
print(final_sparse.size, final_sparse.shape)

13018908 (1084909, 12)
10847587 (1084909, 1126721)


In [ ]:
# can place sparse matrix directly into model data
x_train, x_test, y_train, y_test = train_test_split(final_sparse, y, train_size=.8, random_state = 42)

model = LinearRegression()
model.fit(x_train, y_train)

# predict
y_pred = model.predict(x_test)

# model metrics
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("Root Mean Squared Error:", rmse)
print("R² Score:", r2)

# 

Intercept: 4489014.583747216
Coefficients: [ 3.16215466e+00  1.86926683e+01  6.00869371e+02 ... -1.14599740e+04
 -6.80576813e+03 -2.31927731e+04]
Root Mean Squared Error: 767497.6177269022
R² Score: 0.4675387646050265
